In [2]:
import os
from dotenv import load_dotenv,find_dotenv
_=load_dotenv(find_dotenv())
google_api_key=os.environ['GEMINI_API_KEY']

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",api_key=google_api_key)

#testing
response=llm.invoke("what is 2+3")
print(response.content)

2 + 3 = 5


#Loading PDF Data

In [3]:
!pip install pypdf

  Using cached pypdf-5.1.0-py3-none-any.whl.metadata (7.2 kB)
Using cached pypdf-5.1.0-py3-none-any.whl (297 kB)


In [6]:
from langchain_community.document_loaders import PyPDFLoader
path="Data\safepdf.pdf"
loader=PyPDFLoader(file_path=path)
data=loader.load()

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\panka\AppData\Local\Temp\ipykernel_12892\1347100815.py:2: SyntaxWarning: invalid escape sequence '\s'
  path="Data\safepdf.pdf"


In [9]:
data

[Document(metadata={'source': 'Data\\safepdf.pdf', 'page': 0}, page_content='Rescue Care: Revolutionizing \nAccident and Emergency Response \nwith Smart Assistance\nRescue Care is a comprehensive solution for accidents involving humans or animals, \noffering instant assistance during emergencies and providing peace of mind.'),
 Document(metadata={'source': 'Data\\safepdf.pdf', 'page': 1}, page_content='The Problem:\nRoad accidents are a major global issue, causing severe harm to both humans and \nanimals. After an accident, immediate medical support is crucial in preventing \nfatalities and minimizing long-term damage. Millions of lives could be saved every \nyear if timely intervention like medical services alert system in emergency is ensured. \nOur Proposed solution:\nWe are developing a Smart Alert Platform to provide immediate assistance during \nroad accidents. The platform quickly notifies the nearest health services, primarily \nSadar hospitals, and dispatches ambulances using 

In [27]:
print(data[0].page_content)

Rescue Care: Revolutionizing 
Accident and Emergency Response 
with Smart Assistance
Rescue Care is a comprehensive solution for accidents involving humans or animals, 
offering instant assistance during emergencies and providing peace of mind.


#creating RAG

In [30]:
!pip install -qU langchain-community faiss-cpu

In [13]:
#1 Chunking the Data

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20)
splits=text_splitter.split_documents(data)

In [21]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=google_api_key)
vector = embeddings.embed_query("hello, world!")
vector[:5]


[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [32]:
from langchain_core.vectorstores import InMemoryVectorStore
# Initialize with an embedding model
vector_store = InMemoryVectorStore(embedding=embeddings)
vector_store.add_documents(documents=splits)


['275f65e4-6a24-41e3-ad65-daccb798a8ee',
 '95441c42-8097-4a3b-8b9b-16194b1b6fcb',
 '8d3e69f4-ba20-4424-8f9c-3f3651ce9a07',
 '35cbde94-655f-4883-8d9c-6ce8ee26ffc5',
 '256c27b4-be41-4fa2-8dc4-2c7ff2e3c99c',
 '5c1bf0fa-c28e-4a84-b15a-ba492be01a5f',
 '1580498d-802a-4a8d-8a67-94177700fb5e',
 '5388c8f4-5c96-4622-ad76-a639ae3d28c9',
 '00cd84b8-0669-4066-8438-ede45edb9fee',
 '669006da-9085-4725-847f-787937e2703f',
 'b18bcd02-8dc0-48ca-8d28-fd0defe2a42f',
 '30ef6622-bb7a-47bb-ad44-daab4cb5969f',
 '55faf2a9-61dc-4dfd-bfe8-3941539f812a',
 '8c8ce73c-ca6a-43da-876e-07d29b864722',
 '66169527-6431-4e55-8b43-1808c9071a1e',
 'c88f5409-cd5d-4520-a669-0ca3827a25aa',
 '1cc40e90-6076-448e-b3d8-d4ca23ecaae2',
 '9bf5d5b5-b8cf-40ce-9f0e-81b2c0514d2e',
 '05d4723d-1b36-41fa-b11f-f1da5c80a4fc',
 '7b462fca-4074-41ff-970f-009173489668',
 '71833614-a4fd-4a5f-a827-739e651e3a03',
 'd150e448-b581-49f4-828f-70405b40d7f2',
 'd81a4a99-5834-4f51-a469-b6f8372993e4',
 '60a3bc95-2df9-4baf-9a64-b9c5a568004d',
 'b16acb08-c86d-

In [33]:
retriver=vector_store.as_retriever()

In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

rag_chain = create_retrieval_chain(retriver, question_answer_chain)

results = rag_chain.invoke({"input": "Give the Breif Summay about it and give extact percentage of succes as a startup"})

results["answer"]

"This startup offers unique features like crowd help and animal assistance, differentiating itself in the market.  It also empowers users through technology.  The provided text doesn't mention any success rate."